In [ ]:
! pip install tensorflow numpy glob tqdm scikit-learn Pillow

# Bin image

## Google drive link (Segment BIN images): 
* https://drive.google.com/file/d/1-1DpWRCnXLjVmek41ufd4FWbjdnhJVgy/view?usp=sharing
```same folder
|
├─05_feature_extraction.ipynb
│
├─env
├─aug_seg_bin
├─aug_seg_rgb
|  
```

In [1]:
# ! unzip -q  "/content/drive/MyDrive/Colab Notebooks/Human_Activity_Recognition/aug_seg_bin.zip"

In [4]:
import tensorflow as tf
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelBinarizer
from PIL import Image
import os

In [2]:
IMG_SIZE_H = 480
IMG_SIZE_W = 640

In [5]:
def build_feature_extractor_imagenet():
    feature_extractor = tf.keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE_H, IMG_SIZE_W, 3),
    )
    preprocess_input = tf.keras.applications.inception_v3.preprocess_input
    inputs = tf.keras.Input((IMG_SIZE_H, IMG_SIZE_W, 3))
    preprocessed = preprocess_input(inputs)
    outputs = feature_extractor(preprocessed)
    return tf.keras.Model(inputs, outputs, name="feature_extractor_imgnet")

feature_extractor_imagenet = build_feature_extractor_imagenet()

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
def get_images(img_paths):
    images = []
    for img_path in img_paths:
        img =  Image.open(img_path)
        img = np.asarray(img, dtype=np.float32)
        images.append(img)
        del img
    return np.array(images)

In [7]:
source_dir = "aug_seg_bin"
dist_dir = "aug_feature_seg_bin"

In [11]:
all_img_bin_dirs = glob(f"{source_dir}/*/*")
len(all_img_bin_dirs)

15582

In [10]:
def save_feature(img_paths, features):
    os.makedirs(os.path.dirname(img_paths[0].replace(source_dir, dist_dir)), exist_ok=True)
    for img_path, feature in zip(img_paths, features):
        dits_path = img_path.replace(source_dir, dist_dir).replace('.jpg', '.npy')
        np.save(dits_path, feature)

In [ ]:
for img_dir_path in tqdm(all_img_bin_dirs):
    img_paths = glob(f'{img_dir_path}/*')
    if os.path.exists(img_dir_path.replace(source_dir, dist_dir)):
        continue
    images = get_images(img_paths)
    if len(images.shape) == 3 :
        images = np.expand_dims(images, axis=-1)
        images = np.repeat(images, 3, axis=-1)
    features = feature_extractor_imagenet(images)
    features = features.numpy()
    save_feature(img_paths, features)

In [ ]:
# ! zip -q -r "aug_feature_seg_bin.zip" "aug_feature_seg_bin"

In [ ]:
# ! cp "aug_feature_seg_bin.zip"  "/content/drive/MyDrive/Colab Notebooks/Human_Activity_Recognition/"

# RGB img

## Google drive link (Segment RGB images): 
* https://drive.google.com/file/d/1erUQxsy35emFLkKUKYtwsbvm9xj7yfeF/view?usp=sharing

In [ ]:
# ! unzip -q "/content/drive/MyDrive/Colab Notebooks/Human_Activity_Recognition/aug_seg_rgb.zip"

In [ ]:
import tensorflow as tf
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelBinarizer
from PIL import Image
import os

In [ ]:
IMG_SIZE_H = 480
IMG_SIZE_W = 640

In [ ]:
def build_feature_extractor_imagenet():
    feature_extractor = tf.keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE_H, IMG_SIZE_W, 3),
    )
    preprocess_input = tf.keras.applications.inception_v3.preprocess_input
    inputs = tf.keras.Input((IMG_SIZE_H, IMG_SIZE_W, 3))
    preprocessed = preprocess_input(inputs)
    outputs = feature_extractor(preprocessed)
    return tf.keras.Model(inputs, outputs, name="feature_extractor_imgnet")

feature_extractor_imagenet = build_feature_extractor_imagenet()

In [ ]:
def get_images(img_paths):
    images = []
    for img_path in img_paths:
        img =  Image.open(img_path)
        img = np.asarray(img, dtype=np.float32)
        images.append(img)
        del img
    return np.array(images)

In [ ]:
source_dir = "aug_seg_rgb"
dist_dir = "aug_feature_seg_rgb"

In [ ]:
all_img_rgb_dirs = glob(f"{source_dir}/*/*")
len(all_img_rgb_dirs)

In [ ]:
def save_feature(img_paths, features):
    os.makedirs(os.path.dirname(img_paths[0].replace(source_dir, dist_dir)), exist_ok=True)
    for img_path, feature in zip(img_paths, features):
        dits_path = img_path.replace(source_dir, dist_dir).replace('.jpg', '.npy')
        np.save(dits_path, feature)

In [ ]:
for img_dir_path in tqdm(all_img_rgb_dirs):
    img_paths = glob(f'{img_dir_path}/*')
    if os.path.exists(img_dir_path.replace(source_dir, dist_dir)):
        continue
    images = get_images(img_paths)
    if len(images.shape) == 3 :
        images = np.expand_dims(images, axis=-1)
        images = np.repeat(images, 3, axis=-1)
    features = feature_extractor_imagenet(images)
    features = features.numpy()
    save_feature(img_paths, features)

In [ ]:
# ! zip -q -r "aug_feature_seg_rgb.zip" "aug_feature_seg_rgb"

In [ ]:
# ! cp "aug_feature_seg_rgb.zip"  "/content/drive/MyDrive/Colab Notebooks/Human_Activity_Recognition/"

In [ ]:
import cv2
import numpy as np

def resize_with_padding(image, target_height, target_width):
    # Get the original dimensions
    original_height, original_width = image.shape[:2]

    # Calculate the scaling factor while maintaining aspect ratio
    scale = min(target_width / original_width, target_height / original_height)
    
    # Compute the new size while maintaining the aspect ratio
    new_width = int(original_width * scale)
    new_height = int(original_height * scale)
    
    # Resize the image
    resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)

    # Create a black canvas of the target size
    canvas = np.zeros((target_height, target_width, 3), dtype=np.uint8)

    # Calculate the top-left corner position to center the image on the canvas
    x_offset = (target_width - new_width) // 2
    y_offset = (target_height - new_height) // 2

    # Place the resized image onto the black canvas
    canvas[y_offset:y_offset + new_height, x_offset:x_offset + new_width] = resized_image

    return canvas

# Load your image (replace 'image.jpg' with the path to your image)
# image = cv2.imread('image.jpg')
# make a fake image
image = np.ones((1920,1080, 3), dtype=np.uint8)*255


# Define your target size
target_height1, target_width1 = 250, 100
target_height2, target_width2 = 100, 250

# Resize and pad the image to both target sizes
resized_image1 = resize_with_padding(image, target_height1, target_width1)
resized_image2 = resize_with_padding(image, target_height2, target_width2)

# Save or display the results
cv2.imwrite('resized_250x100.jpg', resized_image1)
cv2.imwrite('resized_100x250.jpg', resized_image2)
cv2.imshow('Resized 250x100', resized_image1)
cv2.imshow('Resized 100x250', resized_image2)
cv2.waitKey(0)
cv2.destroyAllWindows()


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/yato/ML/human-activity-recognition/env/lib/python3.12/site-packages/cv2/qt/plugins"


In [9]:
import numpy as np
a = np.load("../data/keypointDataset/class_01/00000/003.npy")
a

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=int16)

In [11]:
np.load("../data/boxDataset/class_01/00000/003.npy")

array([], dtype=int16)